# Handling a painful CSV file!
Kieran Blacker - 04/12/2020
Using [pandas](https://pandas.pydata.org/). This is a worked solution including my thought process. 

Skip [here](#solution) for the short solution. A .py script is also on my [git](https://github.com/8556732/random/tree/main/Painful%20CSV%20data). 

## Problem statement:
A colleage emailed me saying that they had a problem manipulating some data from the environment agency, and wondered if I could help them get it into a usable format for Excel/MATLAB. 

The email read something like this (edited):

*"I was wondering if you had something to hand that could unravel a dataset from the environment agency...I’m struggling with getting it to do exactly what I want. As such, I was wondering if you had something or could suggest some pointers that might help?*

*I’ve attached a csv. The issue is that every row is a different, what I’d like is to have the variable names as column headers and each row be an individual sample. The challenge is that within the spreadsheet there are multiple samples that have been taken from a range of places, but some of these are also duplicate samples from the same place but taken at different times."*

After reading about the data [here](https://environment.data.gov.uk/water-quality/view/doc/reference) I discovered it was a CSV output from a REST database so we'd need to do some **pivoting**, **joins**, and maybe other stuff. Other than that I came into this pretty naively. 

I originally did this as a script within an IDE, I'm using this notebook to try and recreate my steps and thought process so that you can see what I've done. 

**I am not hosting the data on my git, please access yourself**

### Step 1 - import pandas and load data

In [1]:
import pandas as pd
df = pd.read_csv("DC-2019.csv")

### 2. Initial data exploration
I always like to do this first, you can use the [.head()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html) and [.describe()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) functions of pandas to do some exploration. 

In [2]:
df.head(5) # first 5 lines and columns

,id,sample.samplingPoint,sample.samplingPoint.notation,sample.samplingPoint.label,sample.sampleDateTime,determinand.label,determinand.definition,determinand.notation,resultQualifier.notation,result,codedResultInterpretation.interpretation,determinand.unit.label,sample.sampledMaterialType.label,sample.isComplianceSample,sample.purpose.label,sample.samplingPoint.easting,sample.samplingPoint.northing
0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,SW-60510038,BCOS SEDIMENTS WFD-BI,2019-03-22T10:23:00,NGR Easting,NGR : Easting,6019,NaN,249381.0000,NaN,ngr,COASTAL / MARINE SEDIMENT,False,PLANNED INVESTIGATION (LOCAL MONITORING),263200,153500
1,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,SW-60510038,BCOS SEDIMENTS WFD-BI,2019-03-22T10:23:00,phi 8.5-9.0,Grain Size Fraction : phi 8.5 to 9.0,7486,NaN,0.0366,NaN,%,COASTAL / MARINE SEDIMENT,False,PLANNED INVESTIGATION (LOCAL MONITORING),263200,153500
2,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,SW-60510038,BCOS SEDIMENTS WFD-BI,2019-03-22T10:23:00,phi 4.0-4.5,Grain Size Fraction : phi 4.0 to 4.5,7477,NaN,0.2070,NaN,%,COASTAL / MARINE SEDIMENT,False,PLANNED INVESTIGATION (LOCAL MONITORING),263200,153500
3,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,SW-60510038,BCOS SEDIMENTS WFD-BI,2019-03-22T10:23:00,HTrlTW,Time of sampling relative to previous high water,7342,NaN,3.3200,NaN,hh.mm,COASTAL / MARINE SEDIMENT,False,PLANNED INVESTIGATION (LOCAL MONITORING),263200,153500
4,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,SW-60510038,BCOS SEDIMENTS WFD-BI,2019-03-22T10:23:00,GSF 5.6 - 8,Grain Size Fraction : 5600-8000um : {-2.5 to -...,3669,NaN,5.3400,NaN,%,COASTAL / MARINE SEDIMENT,False,PLANNED INVESTIGATION (LOCAL MONITORING),263200,153500


In [3]:
df.describe() # a summary of the contents as they are currently structured

,determinand.notation,result,codedResultInterpretation.interpretation,sample.samplingPoint.easting,sample.samplingPoint.northing
count,175833.000000,1.758330e+05,0.0,175833.000000,175833.000000
mean,3748.772409,1.659759e+03,NaN,233551.149181,70960.340562
std,3520.023482,5.612835e+04,NaN,48654.392178,30634.704805
min,4.000000,-1.663000e+02,NaN,135560.000000,11700.000000
25%,135.000000,2.900000e-02,NaN,192102.000000,48900.000000
50%,3632.000000,2.730000e+00,NaN,238501.000000,66040.000000
75%,6460.000000,1.400000e+01,NaN,272080.000000,87937.000000
max,9993.000000,9.999999e+06,NaN,345730.000000,153500.000000


#### So a few comments
```id``` looks to be a unique URL to the data stored in the online database, where the url is a combination of sample number, analysis type and location. 

Also - we've got mixed data types in our series as they are currently organised. So mixed numbers and text (strings)

At this point I was unsure what to use as a unique identifier (UID) so I did some more exploration

In [4]:
print(len(pd.unique(df["determinand.notation"]))) 
print(len(pd.unique(df["determinand.label"]))) 
print(len(pd.unique(df["determinand.definition"]))) 

699
697
699


#### uh oh - there's an inconsistency 
I was expecting the number of columns to be all the same, after all if they're all different variables correctly labelled then we should expect all to equal 699 OR 697. My guess is that ```label``` has some repeats or duplicates. Better to stick with notation for now. 

At this point I contacted my colleague and they were fine keeping the column heads as numeric ```notation```

In my original working file I wrote this: 
```
Ok so this returns [699, 697, 699]

This tells me there are 699 unique dtypes but there are duplicate labels
I'm going to group by notation, so columns will have numerical notation

We also need a unique identifier for each point as the id var contains the 
determinand notation, which doesn't help us. 

So I'm going to multiply easting and northing together, to make a UID
```



In [5]:
df['UID'] = pd.Series(df['sample.samplingPoint.easting']*df['sample.samplingPoint.northing'])
# appends UID column to dataframe
print(len(pd.unique(df['UID']))) # check number of unique points = 1695

1695


I then realised that wouldn't work - remember the email:

*"The challenge is that within the spreadsheet there are multiple samples that have been taken from a range of places, but some of these are also duplicate samples from the same place but taken at different times."*

OK - so back to using something unique related to the sample. Again I went ahead and checked what returned 699:

In [6]:
print(len(pd.unique(df["sample.samplingPoint"]))) 
print(len(pd.unique(df["sample.samplingPoint.notation"]))) 
print(len(pd.unique(df["sample.samplingPoint.label"]))) 
print(len(pd.unique(df["sample.sampleDateTime"]))) 

1731
1731
1730
13374


#### ok so number of locations = 1731
We'll use this as the UID

Next step is to transform the dataframe into something that has 1731 rows and 699 columns. 

### 3. Reshaping the dataframe
To turn all unique instances of ```determinand.notation``` into columns we need to unstack the data using a pivot table. 

This is identical in concept to a SQL or Excel pivot, but apparently much [faster](https://pandas.pydata.org/pandas-docs/stable/user_guide/enhancingperf.html?highlight=faster) in pandas. 

**This takes a while to run in a notebook**

In [7]:
df2 = df.pivot_table(index='id', columns=["determinand.notation"], values="result") # this now has text "id" index
df2.head(5)

determinand.notation,4,6,30,48,50,52,61,72,73,76,...,9943,9944,9945,9959,9978,9979,9987,9989,9990,9993
id,,,,,,,,,,,,,,,,,,,,,
http://environment.data.gov.uk/water-quality/data/measurement/SW-3368528-0061,NaN,NaN,NaN,NaN,NaN,NaN,7.73,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
http://environment.data.gov.uk/water-quality/data/measurement/SW-3368528-0076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
http://environment.data.gov.uk/water-quality/data/measurement/SW-3368528-0077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
http://environment.data.gov.uk/water-quality/data/measurement/SW-3368528-0111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
http://environment.data.gov.uk/water-quality/data/measurement/SW-3368528-0116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### why all the NaN's?

So when we pivoted I did not give any instruction on how to aggregate the data. The default in pandas is ```None``` which is good for us - we want to keep all the data columns, but remove the duplicate rows. 

The only issue now is that we've lost a numeric index, but we can add that back, but also get rid of that horrible URL ```id``` field:

In [8]:
df2 = df2.reset_index(drop=True)

Ok so now we need all the points, with the same index as the new dataframe so that we get all the repeats. 

Well that's easy, because we did not aggregate the data, so the original index will have been preserved and then reassigned as numeric in the last statement. 

We can extract the column (series) of points as a single column dataframe:

In [9]:
pointID = df["sample.samplingPoint.notation"]

We then do a full join on the numeric index of our new ```pointID``` frame:

In [10]:
df2 = df2.join(pointID)

Let's check what we've done again:

In [11]:
df2.head(n=5)

,4,6,30,48,50,52,61,72,73,76,...,9944,9945,9959,9978,9979,9987,9989,9990,9993,sample.samplingPoint.notation
0,NaN,NaN,NaN,NaN,NaN,NaN,7.73,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SW-60510038
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SW-60510038
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SW-60510038
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SW-60510038
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SW-60510038


Ok that looks good. We've got the variables and the point labels. 
### 4. Removing duplicates:
But we've got no way to tell pandas how to remove the duplicates - we need a dummy field or fields to melt the dataframe by each individual point. 

We also want the coordinates and other columns back - so we need another join. 

But before we can do that we need to handle the NaN values. There's a few ways we could do this, but as we are going to melt the duplicate rows, the simplest method is to just groupby and sum them before we drop the duplicates:

In [12]:
df2c = df2.groupby(['sample.samplingPoint.notation']).sum(min_count=1)

We can now drop all the duplicates in our original dataframe, and reindex the data to ```sample.samplingPoint.notation``` using:

In [13]:
dfc = df.drop_duplicates(subset='sample.samplingPoint.notation') # drop all repeats from original
dfc = dfc.set_index('sample.samplingPoint.notation') # set sample id as index for join

Now the final step - full join of the two dataframes:

In [14]:
merge = dfc.join(df2c)

Check the result:

In [15]:
merge.head(n=5)

,id,sample.samplingPoint,sample.samplingPoint.label,sample.sampleDateTime,determinand.label,determinand.definition,determinand.notation,resultQualifier.notation,result,codedResultInterpretation.interpretation,...,9943,9944,9945,9959,9978,9979,9987,9989,9990,9993
sample.samplingPoint.notation,,,,,,,,,,,,,,,,,,,,,
SW-60510038,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,BCOS SEDIMENTS WFD-BI,2019-03-22T10:23:00,NGR Easting,NGR : Easting,6019,NaN,249381.000,NaN,...,0.820,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SW-7002GW02,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,"WOODHEAD FARM, BRANSCOMBE",2019-03-13T11:55:00,Orthophospht,"Orthophosphate, reactive as P",180,NaN,0.044,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SW-7002GW04,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,RINGBOROUGH HOUSE,2019-01-11T12:54:00,Orthophospht,"Orthophosphate, reactive as P",180,<,0.010,NaN,...,4.037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SW-7002GW06,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,PENDRAGON FARM,2019-01-07T12:47:00,Chloride Ion,Chloride,172,NaN,13.400,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SW-7002GW07,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,CRICKET ST THOMAS,2019-01-07T11:46:00,Orthophospht,"Orthophosphate, reactive as P",180,NaN,0.083,NaN,...,4.480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
merge.describe()

,determinand.notation,result,codedResultInterpretation.interpretation,sample.samplingPoint.easting,sample.samplingPoint.northing,UID,4,6,30,48,...,9943,9944,9945,9959,9978,9979,9987,9989,9990,9993
count,1731.000000,1.731000e+03,0.0,1731.000000,1731.000000,1.731000e+03,236.000000,239.000000,12.000000,1.0,...,271.000000,11.000000,11.000000,2.900000e+01,2.900000e+01,2.600000e+01,1.0,13.000000,9.000,206.000000
mean,2824.410168,3.251839e+03,NaN,238589.305026,75569.442519,1.888506e+10,30.149407,4.957155,0.507500,625.0,...,1.757005,232.781818,295.909091,4.000000e-03,2.000000e-03,5.000000e-03,63.9,0.332154,0.001,0.061711
std,3274.461800,7.390996e+04,NaN,48043.086396,30836.656318,9.677068e+09,25.252015,15.308859,0.361691,NaN,...,2.761970,219.409944,284.220145,2.648143e-18,1.324072e-18,8.845389e-19,NaN,0.742507,0.000,0.089967
min,4.000000,0.000000e+00,NaN,135560.000000,11700.000000,2.446470e+09,4.090000,0.200000,0.100000,625.0,...,0.005000,48.000000,61.000000,4.000000e-03,2.000000e-03,5.000000e-03,63.9,0.002000,0.001,0.002000
25%,116.000000,2.735000e-01,NaN,201260.000000,52196.500000,1.164083e+10,11.280000,0.200000,0.187500,625.0,...,0.122000,49.000000,66.500000,4.000000e-03,2.000000e-03,5.000000e-03,63.9,0.002000,0.001,0.012450
50%,180.000000,4.000000e+00,NaN,241890.000000,71950.000000,1.714772e+10,21.840000,0.600000,0.515000,625.0,...,0.667000,150.000000,180.000000,4.000000e-03,2.000000e-03,5.000000e-03,63.9,0.002000,0.001,0.027500
75%,4883.000000,1.695000e+01,NaN,277384.000000,99000.000000,2.613986e+10,40.147500,1.000000,0.822500,625.0,...,2.575150,373.000000,445.500000,4.000000e-03,2.000000e-03,5.000000e-03,63.9,0.100000,0.001,0.067075
max,9993.000000,3.000000e+06,NaN,345730.000000,153500.000000,4.158436e+10,152.570000,138.450000,0.910000,625.0,...,30.790000,657.000000,827.000000,4.000000e-03,2.000000e-03,5.000000e-03,63.9,2.000000,0.001,0.885000


In [17]:
merge.shape

(1731, 716)

### 5. Final cleaning

We've done it! We've got the right number of rows. But we've also now got some redundant columns. The columns with url's or unique values don't mean anything anymore, so we can drop those: 

In [18]:
merge = merge.reset_index(drop=True)
finaldata = merge.drop(columns=['id','sample.samplingPoint','determinand.label','UID',\
                                'resultQualifier.notation','result','codedResultInterpretation.interpretation',\
                               'determinand.definition', 'determinand.notation', 'determinand.unit.label'])
finaldata.shape
finaldata.head(n=1)

,sample.samplingPoint.label,sample.sampleDateTime,sample.sampledMaterialType.label,sample.isComplianceSample,sample.purpose.label,sample.samplingPoint.easting,sample.samplingPoint.northing,4,6,30,...,9943,9944,9945,9959,9978,9979,9987,9989,9990,9993
0,BCOS SEDIMENTS WFD-BI,2019-03-22T10:23:00,COASTAL / MARINE SEDIMENT,False,PLANNED INVESTIGATION (LOCAL MONITORING),263200,153500,NaN,NaN,NaN,...,0.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<a id='solution'></a>
## The short solution:

All of the above in a single cell. 

In [20]:
import pandas as pd
df = pd.read_csv("DC-2019.csv")
df2 = df.pivot_table(index='id', columns=["determinand.notation"], values="result")
df2 = df2.reset_index(drop=True)
pointID = df["sample.samplingPoint.notation"]
df2 = df2.join(pointID)
df2c = df2.groupby(['sample.samplingPoint.notation']).sum(min_count=1)
dfc = df.drop_duplicates(subset='sample.samplingPoint.notation')
dfc = dfc.set_index('sample.samplingPoint.notation')
merge = dfc.join(df2c)
merge = merge.reset_index(drop=True)
finaldata = merge.drop(columns=['id','sample.samplingPoint','determinand.label',\
                                'resultQualifier.notation','result','codedResultInterpretation.interpretation',\
                               'determinand.definition', 'determinand.notation', 'determinand.unit.label'])
finaldata.to_csv("DC-2019_formatted.csv")

#### Questions or advice or improvements:
